## install prerequisite

In [1]:
from utility.preprocessing1 import processing,load_pickle,get_augmentaion,train_test_split
from models.model1 import padding,train_model,load_model,infer,DiagnosisDataset

cpu


In [2]:
DATA_SIZE=10000
BASE_PATH=f'data/{DATA_SIZE}'
FILE = f"{BASE_PATH}/AdmissionsDiagnosesCorePopulatedTable.txt"

## Run if you want to train your model with new data

In [52]:
# processing(FILE,DATA_SIZE)

## Load data from pickle

In [53]:
data_int,int2token,token2int=load_pickle(DATA_SIZE)

Load data from pickle
length=10000


## if you need augmentaion

In [55]:
data_aug=get_augmentaion(data_int)

Total=59050


## Train test split

In [56]:
train, val = train_test_split(data_aug,ratio=0.05,random_seed=10)
print(f"train: {len(train)} ,val: {len(val)}")

train: 56168 ,val: 2952


# Train model

In [62]:
n_feature=16
n_hidden=128 
n_layer=1
drop_prob=0.10 
batch_size=32
input_size=11
num_epoch = 150
pad_value=2625 

save_path=f"save_model/latest-b{batch_size}-e{num_epoch}_model.pth"

train_model(
    n_feature=n_feature,
    n_hidden=n_hidden,
    n_layer=n_layer,
    drop_prob=drop_prob,
    batch_size=batch_size,
    input_size=input_size,
    num_epoch=num_epoch,
    pad_value=pad_value,
    train=train,
    val=val,
    save_path=save_path
)

## Load model 

In [3]:
DATA_SIZE=10000
data_int,int2token,token2int=load_pickle(DATA_SIZE)
train, val = train_test_split(data_int,ratio=0.0,random_seed=10)

n_feature=16
n_hidden=128
n_layer=1
drop_prob=0.10
pad_value=2625
input_size=11

save_path="save_model/latest_32_model.pth"
model=load_model(n_feature,n_hidden,n_layer,drop_prob,save_path)

Load data from pickle
length=10000
Model Loaded


## Evaluation

In [5]:

import random
import torch
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

eval_data = DiagnosisDataset(train,11,pad_value=2625)

random.seed(1)
list_rand = random.sample(range(8000),8000)
correct = []
wrong = []
for idx in list_rand:
    ip = torch.from_numpy(eval_data[idx][0]).view(1,-1)
    gt = eval_data[idx][1]
    with torch.no_grad():
        y_hat, _ = model(ip.to(device))
        y_hat = F.softmax(y_hat,1).cpu()
        _, indx = torch.max(y_hat,1)
        if indx.item() == gt:
            correct.append(gt)
        else:
            wrong.append({"true":gt, "predicted":indx.item(),"index":idx})
total = len(correct) + len(wrong)
print(f"accuracy: {len(correct)/total}")


accuracy: 0.748


## Test model


In [6]:

def predict():
    num=int(input("How many Diagnoses code do you have for next code prediction ?\n"))
    x_test=[]
    for x in range(num):
        x_code=input(f"Enter Diagnoses code {x+1} = ").strip().upper()
        try:
            x_test.append(token2int[x_code])
        except:
            print("Embedding not present")
    x=[int2token[x] for x in x_test]
    x_test=padding(x_test,input_size,pad_value)
    idy=infer(x_test,model)
    y=int2token[idy]
    print("\n........Prediction........\n")
    print(f"{x} --> {y}")

predict()

How many Diagnoses code do you have for next code prediction ?
 4
Enter Diagnoses code 1 =  h93.25
Enter Diagnoses code 2 =  o99.285
Enter Diagnoses code 3 =  c79.0
Enter Diagnoses code 4 =  q21



........Prediction........

['H93.25', 'O99.285', 'C79.0', 'Q21'] --> Z13.0


In [30]:
[int2token[x] for x in train[73]]

['H93.25', 'O99.285', 'C79.0', 'Q21', 'Z13.0']

In [63]:
# p=['F32.4', 'Q27.1', 'H05.321', 'M31.30']